In [3]:
#parameters
experimental_datafile = 'inputs/results.csv'
domain_configfile = 'inputs/borrowing_hydrogen_domain.json'
batch_size = 10

In [10]:
# from summit.strategies import TSEMO
from summit.models import GPyModel

from gp_optimizer import DirectOpt
# from summit.optimizers import EnumerationOptimizer
# from summit.objective import HV
# from summit.domain import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Import data
domain = Domain.from_configfile(domain_configfile)
previous_results = DataSet.from_csv(experimental_datafile)

In [ ]:
#Run the optimization
kernel = GPy.kern.Matern52(input_dim = domain.num_continuous_dimension+domain.num_discrete_variables, ARD=TRUE)
models = [GPyModel(kernel=kernel) for _ in range(2)]
tsemo = TSEMO(domain, models, acquisition=HV(), optimizer=EnumerationOptimizer())
design = tsemo.generate_experiments(previous_results, batch_size)

In [ ]:
#Generate some nice plots

##3d plots of the GP models and original data in 2D pc space

##Plots of the models' hyperparameters for each batch including this one

##Plots of the models' log likelihood after each batch including this one

##Plots of the hypervolume for each batch inlcuding this one

In [ ]:
#Save design for next experiment(s) and model hyperparameters to disk                                     
next_batch_num = previous_results['batch_number'].max() + 1
design.insert(0, 'batch_number', np.ones(design.shape[0])*next_batch_num, type='METADATA')
design.to_csv(f'outputs/batch_{next_batch_num}.csv')
tsemo.save_params(f'outputs/{batch}')